### 포켓몬 이미지 수집을 위한 파이썬 코드
- 웹 크롤링을 통한 이미지 파일의 수집
- 폴더 생성
- 이미지 링크 수집 후 다운로드 자동화

In [192]:
# 필요한 라이브러리 import

import os  # 운영체제 관련 작업 (폴더 생성, 삭제 등)
import time  # 시간 관련 기능 (대기, 지연)

try:
    from selenium import webdriver as wb
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
except ModuleNotFoundError:
    !pip install selenium

In [193]:
# 필요한 것들 정리

# 포켓몬 이미지 파일의 위치
POKEMON_URL = 'https://www.pokemonkorea.co.kr/pokedex#pokedex_1'

# 메인폴더 위치
ORIGIN_URL = "C:\\Users\\smhrd1\\Documents\\GitHub\\Crawling\\Study"

# 생성할 폴더의 이름
MKDIR_NAME = 'pokemon'

# 1,225개?의 이미지가 나올 것
# class = 'img-fluid' 인 img를 크롤링
TARGET_IMG_CLASS = 'img-fluid'
TARGET_NAME_CLASS = 'div.bx-txt > h3'

In [194]:
os.chdir(ORIGIN_URL)

print(f'현재 위치 : {os.getcwd()}')

try :
    os.mkdir(MKDIR_NAME) 
    print(f'{MKDIR_NAME}폴더를 생성했습니다.')
except FileExistsError:
    print("메인 폴더가 이미 존재합니다.") 
except :
    print("폴더 생성 에러")
 
try : 
    os.chdir(MKDIR_NAME)
    print(f'{MKDIR_NAME}폴더로 이동합니다.')
except NameError :
    print('이동할 폴더가 없습니다.')

print(f'현재 위치 : {os.getcwd()}')

index = 1
while(True):
    index_str = str(index)  # 정수를 문자열로 변환
    if not os.path.isdir(index_str):
        os.mkdir(index_str)
        print(f'{index_str} 폴더를 생성했습니다.')
        break
    else:
        index += 1

try:
    os.chdir(index_str)
    print(f'{index_str} 폴더로 이동합니다.')
except OSError:
    print('이동할 폴더가 없습니다.')

print(f'현재 위치 : {os.getcwd()}')

현재 위치 : C:\Users\smhrd1\Documents\GitHub\Crawling\Study
메인 폴더가 이미 존재합니다.
pokemon폴더로 이동합니다.
현재 위치 : C:\Users\smhrd1\Documents\GitHub\Crawling\Study\pokemon
5 폴더를 생성했습니다.
5 폴더로 이동합니다.
현재 위치 : C:\Users\smhrd1\Documents\GitHub\Crawling\Study\pokemon\5


In [195]:
# 크롬 실행
driver = wb.Edge()
driver.get(POKEMON_URL)
time.sleep(3)

In [196]:
# selenium 을 통해 스크롤을 내릴 수 없음. 그럼 어찌 해야 되나??
# 대안 : 키보드의 end키를 눌러야 됨.

 
while(True) :  
    body = driver.find_element(By.TAG_NAME, 'body')
    before_find = len(driver.find_elements(By.CSS_SELECTOR, 'li')) 
    body.send_keys(Keys.END)
    time.sleep(0.5) 
    later_find = len(driver.find_elements(By.CSS_SELECTOR, 'li')) 
    
    if (before_find == later_find): 
        # 느린 인터넷의 에러를 보정
        print(f'페이지가 끝났는지 검사 중... ')
        time.sleep(2) 
        later_find_check = len(driver.find_elements(By.CSS_SELECTOR, 'li')) 
        if(later_find == later_find_check) :
            print(f'페이지가 끝나서 반복을 종료합니다.')
            break 
        else :
            print(f'페이지가 끝나지 않았음')

페이지가 끝났는지 검사 중... 
페이지가 끝나서 반복을 종료합니다.


In [197]:
get_All_IMG = driver.find_elements(By.CLASS_NAME, TARGET_IMG_CLASS)

temp_get_All_info = driver.find_elements(By.CSS_SELECTOR, TARGET_NAME_CLASS)  

In [198]:
# 반복분  
try : 
    from tqdm import tqdm
except ModuleNotFoundError :
    !pip install tqdm

In [200]:
# 속성을 가지고 오는 함수
# get_attribute('')  

url_list = [i.get_attribute('src') for i in tqdm(get_All_IMG)]

temp_get_All_Name = [name.text.split('\n')[1] for name in temp_get_All_info] 
temp_get_All_Number = [number.text.split('\n')[0] for number in temp_get_All_info]

name_list = [name.replace("'", "") for name in tqdm(temp_get_All_Name)] 
number_list = [name.replace(".", "_") for name in tqdm(temp_get_All_Number)]  

100%|█████████████████████████████████████████████████████████████████████████| 1252/1252 [00:00<00:00, 3892712.09it/s]


In [201]:
driver.close()

In [202]:
print(f'이미지 수 : {len(url_list)}')
print(f'이름의 수 : {len(name_list)}')
print(f'도감의 수 : {len(number_list)}')

이미지 수 : 1252
이름의 수 : 1252
도감의 수 : 1252


In [203]:
# url을 활용하여 이미지 다운로들
from urllib.request import urlretrieve

In [205]:
# urlretrieve('이미지 링크', '경로 파일명.확장자')  
for i in tqdm(range(len(url_list))):
    file_order = 0
    url = url_list[i]  
    file_name = f"{number_list}_{name_list[i]}_{file_order:02}.jpg"   
    while(True) : 
        check = os.path.isfile(file_name)
        #만약 이미 있으면 true
        if(check) : 
            file_order += 1
            file_name = f"{number_list}_{name_list[i]}_{file_order:02}.jpg"    
        else :
            break
    urlretrieve(url, file_name)    
print(f'저장 완료!')

  0%|                                                                                         | 0/1252 [00:00<?, ?it/s]


OSError: [Errno 22] Invalid argument: "['No_0001', 'No_0002', 'No_0003', 'No_0003', 'No_0003', 'No_0004', 'No_0005', 'No_0006', 'No_0006', 'No_0006', 'No_0006', 'No_0007', 'No_0008', 'No_0009', 'No_0009', 'No_0009', 'No_0010', 'No_0011', 'No_0012', 'No_0012', 'No_0013', 'No_0014', 'No_0015', 'No_0015', 'No_0016', 'No_0017', 'No_0018', 'No_0018', 'No_0019', 'No_0019', 'No_0020', 'No_0020', 'No_0021', 'No_0022', 'No_0023', 'No_0024', 'No_0025', 'No_0025', 'No_0026', 'No_0026', 'No_0027', 'No_0027', 'No_0028', 'No_0028', 'No_0029', 'No_0030', 'No_0031', 'No_0032', 'No_0033', 'No_0034', 'No_0035', 'No_0036', 'No_0037', 'No_0037', 'No_0038', 'No_0038', 'No_0039', 'No_0040', 'No_0041', 'No_0042', 'No_0043', 'No_0044', 'No_0045', 'No_0046', 'No_0047', 'No_0048', 'No_0049', 'No_0050', 'No_0050', 'No_0051', 'No_0051', 'No_0052', 'No_0052', 'No_0052', 'No_0052', 'No_0053', 'No_0053', 'No_0054', 'No_0055', 'No_0056', 'No_0057', 'No_0058', 'No_0058', 'No_0059', 'No_0059', 'No_0060', 'No_0061', 'No